<a href="https://colab.research.google.com/github/AndreyV80/NetStudy/blob/main/Advanced_pandas_ProjectWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np


import os, time
from google.colab import drive

In [2]:
drive.mount('/content/drive')    


Mounted at /content/drive


**Задание 1**

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

если источник traffic_source равен yandex или google, то в source_type ставится organic
для источников paid и email из России - ставим ad
для источников paid и email не из России - ставим other
все остальные варианты берем из traffic_source без изменений

In [ ]:
visit_log = pd.read_csv('/content/drive/My Drive/visit_log.csv', sep = ';')
visit_log.head()


,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [ ]:

log = visit_log.copy()

In [ ]:
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [ ]:
traffic_source_set = set(log['traffic_source'])  #просматриваем уникальные значения в столбце "traffic_source"
traffic_source_set


{'direct', 'email', 'google', 'paid', 'yandex'}

In [ ]:
# из материалов занятия, использую .loc, формируем столбец "source_type" со значениями согласно условиям
log.loc[(log.traffic_source == 'yandex') | (log.traffic_source == 'google'), 'source_type'] = 'organic' 
log.loc[(log.region == 'Russia') & ((log.traffic_source == 'paid') | (log.traffic_source == 'email')), 'source_type'] = 'ad'
log.loc[(log.region != 'Russia') & ((log.traffic_source == 'paid') | (log.traffic_source == 'email')), 'source_type'] = 'other'

In [ ]:
#log.loc[(log.region == 'Russia') & ((log.traffic_source == 'paid') | (log.traffic_source == 'email')), 'source_type'] = 'ad'
#log.loc[(log.region != 'Russia') & ((log.traffic_source == 'paid') | (log.traffic_source == 'email')), 'source_type'] = 'other'

In [ ]:
log

In [ ]:
log.fillna({'source_type': log.traffic_source}, inplace=True) # поскольку по условиям задачи значения проставлены, остальные ячейки содержат пустоты, заменяем пустоты на значение "traffic_source" 
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


**Задание 2**

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

1. Прочитайте содержимое файла с датафрейм
2. Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [ ]:
import re

In [ ]:
URLs = pd.read_csv('/content/drive/My Drive/URLs.txt')
URLs.head(20)

следующую конструкцию: /, затем 8 цифр, затем дефис

In [ ]:
URLs_news = URLs[URLs.url.str.contains(r'(/\d{8}-)')]

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [ ]:
URLs_news

**Задание 3**

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [3]:
ratings = pd.read_csv('/content/drive/My Drive/ratings.csv')

1. Проверка датафрэйма

In [ ]:
ratings.head() 

In [ ]:
ratings.info()

2. Копируем фрэйм для работы

In [4]:
rat2 = ratings.copy()

3. Группировка по **userId**, определяем максимум, минимум и кол-во по **timestamp**. сохраняем в отдеоьные фрэймы с индексом по столбцу userId

In [6]:
rat2_min = rat2.groupby('userId').min()[['timestamp']]    #.reindex()
rat2_max = rat2.groupby('userId').max()[['timestamp']]     #.reindex()
rat2_count = rat2.groupby('userId').count()[['rating']]


4. Новый фрэйм - объединяем фрэйм кол-ва "rat2_count" с разницей фрэймов "rat2_max" и "rat2_min"  по индексу.

In [7]:
df = rat2_count.merge(rat2_max.sub(rat2_min, axis=1), left_index=True, right_index=True)

In [ ]:
df.head(2) 

,rating,timestamp
userId,,
1,232,739163
2,29,505


5. Фильтруем фрэйм по условию "свыше 100" и сортируем по убыванию

In [9]:
df_sto = df.query('rating > 100').sort_values(by='timestamp', ascending=False)

df_sto.head()       

,rating,timestamp
userId,,
414,2698,566541856
292,446,539729459
448,1864,480496843
199,363,460937018
462,455,414546374


**Задание 4**

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента

***Обратите внимание, что в процессе объединения таблиц данные не должны теряться.***

Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки

auto - автомобильные перевозки

air - воздушные перевозки

client_base - адреса клиентов

In [5]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [6]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [7]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [8]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


Копируем в новый фрейм

In [9]:
client_ = client_base.copy()

In [ ]:
Убираем лишний столбец 'address'

In [10]:
client_base_only = client_.drop(['address'], axis=1)
client_base_only


,client_id
0,111
1,112
2,113
3,114
4,115
5,116
6,117
7,118


Фрейм клиент-выручка-без адреса

In [12]:
client_dohod = client_base_only.merge(air, how='left', left_on='client_id', right_on='client_id').merge(auto,  how='left', left_on='client_id', right_on='client_id').merge(rzd,  how='left', left_on='client_id', right_on='client_id').fillna(0)
client_dohod

,client_id,air_revenue,auto_revenue,rzd_revenue
0,111,0.0,0.0,1093.0
1,112,0.0,0.0,2810.0
2,113,0.0,57483.0,10283.0
3,114,0.0,83.0,5774.0
4,115,81.0,912.0,981.0
5,116,4.0,4834.0,0.0
6,117,13.0,98.0,0.0
7,118,173.0,0.0,0.0


Фрейм клиент-выручка-с адресом

In [13]:
client_dohod_adress = client_base.merge(air, how='left', left_on='client_id', right_on='client_id').merge(auto,  how='left', left_on='client_id', right_on='client_id').merge(rzd,  how='left', left_on='client_id', right_on='client_id').fillna(0)
client_dohod_adress

,client_id,address,air_revenue,auto_revenue,rzd_revenue
0,111,Комсомольская 4,0.0,0.0,1093.0
1,112,Энтузиастов 8а,0.0,0.0,2810.0
2,113,Левобережная 1а,0.0,57483.0,10283.0
3,114,Мира 14,0.0,83.0,5774.0
4,115,ЗЖБИиДК 1,81.0,912.0,981.0
5,116,Строителей 18,4.0,4834.0,0.0
6,117,Панфиловская 33,13.0,98.0,0.0
7,118,Мастеркова 4,173.0,0.0,0.0


In [ ]:
Грубо, но другого не придумал